## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [3]:
# Find all categories with only 2 unique values as can be binary encoded
def convert_yes_no(value):
    if str(value).lower() in ["yes", "y", "true"]:
        return 1
    elif str(value).lower() in ["no", "n", "false"]:
        return 0
    else:
        return value  # return original value if it's neither 'yes' nor 'no'

def process_column(column):
    unique_values = column.dropna().unique()
    if len(unique_values) == 2:
        lowercased_values = set(map(lambda x: str(x).lower(), unique_values))
        if lowercased_values.issubset({"yes", "no", "y", "n", "true", "false"}):
            print(column.name, "is being binary encoded")
            return column.apply(convert_yes_no)
    return column

def convert_yes_no_columns(df):
    for column in df.columns:
        df[column] = process_column(df[column])
    return df

attrition_df = convert_yes_no_columns(attrition_df)
attrition_df.head()

Attrition is being binary encoded
OverTime is being binary encoded


,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,1,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,0,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,1,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,0,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,0,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [4]:
attrition_df["Department"].value_counts()

Department
Research & Development    961
Sales                     446
Human Resources            63
Name: count, dtype: int64

In [5]:
# ToDo Encode this differently (ordinally?)
attrition_df["Attrition"].value_counts()

Attrition
0    1233
1     237
Name: count, dtype: int64

attrition_df.columns

In [6]:
# Convert your X data to numeric data types however you see fit

In [7]:
# Using these target columns later
target_columns = ['Attrition', 'Department']

In [8]:
# Get the columns with data types other than 'int64'
non_int_columns = attrition_df.select_dtypes(exclude='int64').columns.tolist()
print(non_int_columns)

['BusinessTravel', 'Department', 'EducationField', 'JobRole', 'MaritalStatus']


In [9]:
# Important for later
def filter_target_from_non_int(array1, array2):
    return [value for value in array1 if value not in array2]

non_int_columns = filter_target_from_non_int(non_int_columns, target_columns)

In [10]:
from sklearn.preprocessing import OneHotEncoder

# Initialize OneHotEncoder because categories are not ordinal
encoder = OneHotEncoder()
# Extract the columns to be encoded from your DataFrame
columns_to_encode_df = attrition_df[non_int_columns]

In [11]:
# Fit and transform the categorical column
encoded_features = encoder.fit_transform(columns_to_encode_df)

In [12]:
# Convert the encoded features to a DataFrame
encoded_df = pd.DataFrame(encoded_features.toarray(), columns=encoder.get_feature_names_out(non_int_columns))
encoded_df.head()

,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,EducationField_Human Resources,EducationField_Life Sciences,EducationField_Marketing,EducationField_Medical,EducationField_Other,EducationField_Technical Degree,JobRole_Healthcare Representative,...,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single
0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [13]:
# Concatenate the encoded DataFrame with the original DataFrame
attrition_df_encoded = pd.concat([attrition_df.drop(non_int_columns, axis=1), encoded_df], axis=1)
attrition_df_encoded.head()

,Age,Attrition,Department,DistanceFromHome,Education,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,...,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single
0,41,1,Sales,1,2,2,94,3,2,4,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,49,0,Research & Development,8,1,3,61,2,2,2,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,37,1,Research & Development,2,2,4,92,2,1,3,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,33,0,Research & Development,3,4,4,56,3,1,3,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,27,0,Research & Development,2,1,1,40,3,1,2,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [14]:
attrition_df_encoded.columns

Index(['Age', 'Attrition', 'Department', 'DistanceFromHome', 'Education',
       'EnvironmentSatisfaction', 'HourlyRate', 'JobInvolvement', 'JobLevel',
       'JobSatisfaction', 'NumCompaniesWorked', 'OverTime',
       'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction',
       'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear',
       'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole',
       'YearsSinceLastPromotion', 'YearsWithCurrManager',
       'BusinessTravel_Non-Travel', 'BusinessTravel_Travel_Frequently',
       'BusinessTravel_Travel_Rarely', 'EducationField_Human Resources',
       'EducationField_Life Sciences', 'EducationField_Marketing',
       'EducationField_Medical', 'EducationField_Other',
       'EducationField_Technical Degree', 'JobRole_Healthcare Representative',
       'JobRole_Human Resources', 'JobRole_Laboratory Technician',
       'JobRole_Manager', 'JobRole_Manufacturing Director',
       'JobRole_Research Director', 'Jo

In [15]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df_encoded[target_columns]

In [16]:
# Create a list of at least 10 column names to use as X data
copy_df = attrition_df_encoded.copy()
copy_df = copy_df.drop(target_columns, axis=1)
X_columns = copy_df.columns

# Create X_df using your selected columns
X_df = attrition_df_encoded[X_columns]

# Show the data types for X_df
print(X_df.dtypes)


Age                                    int64
DistanceFromHome                       int64
Education                              int64
EnvironmentSatisfaction                int64
HourlyRate                             int64
JobInvolvement                         int64
JobLevel                               int64
JobSatisfaction                        int64
NumCompaniesWorked                     int64
OverTime                               int64
PercentSalaryHike                      int64
PerformanceRating                      int64
RelationshipSatisfaction               int64
StockOptionLevel                       int64
TotalWorkingYears                      int64
TrainingTimesLastYear                  int64
WorkLifeBalance                        int64
YearsAtCompany                         int64
YearsInCurrentRole                     int64
YearsSinceLastPromotion                int64
YearsWithCurrManager                   int64
BusinessTravel_Non-Travel            float64
BusinessTr

In [17]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=42)


In [18]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [19]:
print(y_train["Department"])

1343    Research & Development
1121                     Sales
1048                     Sales
1393                     Sales
527                      Sales
                 ...          
1130    Research & Development
1294    Research & Development
860     Research & Development
1459    Research & Development
1126                     Sales
Name: Department, Length: 1102, dtype: object


In [20]:
# Create a OneHotEncoder for the Department column
one_hot_encoder_department = OneHotEncoder()

# Fit the encoder to the training data
one_hot_encoder_department.fit(y_train["Department"].values.reshape(-1, 1))

# Create two new variables by applying the encoder
# to the training and testing data
y_train_department_encoded = one_hot_encoder_department.transform(y_train["Department"].values.reshape(-1, 1)).toarray()
y_test_department_encoded = one_hot_encoder_department.transform(y_test["Department"].values.reshape(-1, 1)).toarray()

In [21]:
# Create a OneHotEncoder for the Attrition column
one_hot_encoder_attrition = OneHotEncoder()

# Fit the encoder to the training data
one_hot_encoder_attrition.fit(y_train["Attrition"].values.reshape(-1,1))

# Create two new variables by applying the encoder
# to the training and testing data
y_train_attrition_encoded = one_hot_encoder_attrition.transform(y_train["Attrition"].values.reshape(-1, 1))
y_test_attrition_encoded = one_hot_encoder_attrition.transform(y_test["Attrition"].values.reshape(-1, 1))

## Create, Compile, and Train the Model

In [22]:
train_df = pd.DataFrame(X_train)
train_df.head()

,Age,DistanceFromHome,Education,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,NumCompaniesWorked,OverTime,...,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single
1343,29,7,3,4,59,3,1,1,3,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1121,36,1,4,2,73,3,2,3,6,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1048,34,3,3,4,81,1,2,1,3,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1393,27,9,3,4,44,3,2,4,1,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
527,32,10,3,4,55,3,2,4,1,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [23]:
train_df.columns

Index(['Age', 'DistanceFromHome', 'Education', 'EnvironmentSatisfaction',
       'HourlyRate', 'JobInvolvement', 'JobLevel', 'JobSatisfaction',
       'NumCompaniesWorked', 'OverTime', 'PercentSalaryHike',
       'PerformanceRating', 'RelationshipSatisfaction', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager', 'BusinessTravel_Non-Travel',
       'BusinessTravel_Travel_Frequently', 'BusinessTravel_Travel_Rarely',
       'EducationField_Human Resources', 'EducationField_Life Sciences',
       'EducationField_Marketing', 'EducationField_Medical',
       'EducationField_Other', 'EducationField_Technical Degree',
       'JobRole_Healthcare Representative', 'JobRole_Human Resources',
       'JobRole_Laboratory Technician', 'JobRole_Manager',
       'JobRole_Manufacturing Director', 'JobRole_Research Director',
       'JobRole_Research Scientist', 

In [24]:
train_df.nunique()

Age                                  43
DistanceFromHome                     29
Education                             5
EnvironmentSatisfaction               4
HourlyRate                           71
JobInvolvement                        4
JobLevel                              5
JobSatisfaction                       4
NumCompaniesWorked                   10
OverTime                              2
PercentSalaryHike                    15
PerformanceRating                     2
RelationshipSatisfaction              4
StockOptionLevel                      4
TotalWorkingYears                    40
TrainingTimesLastYear                 7
WorkLifeBalance                       4
YearsAtCompany                       34
YearsInCurrentRole                   19
YearsSinceLastPromotion              16
YearsWithCurrManager                 17
BusinessTravel_Non-Travel             2
BusinessTravel_Travel_Frequently      2
BusinessTravel_Travel_Rarely          2
EducationField_Human Resources        2


In [25]:
from tensorflow.keras.layers import Input, Dense
# Find the number of columns in the X training data
num_input_features = len(train_df.columns)
print(num_input_features)
# Create the input layer
input_layer = Input(shape=(num_input_features,))
# Create at least two shared layers
shared_layer1 = Dense(64, activation='relu')(input_layer)
shared_layer2 = Dense(32, activation='relu')(shared_layer1)

42


In [26]:
num_department_classes = attrition_df["Department"].value_counts()
print(len(num_department_classes))

3


In [27]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden_layer = Dense(64, activation='relu')(shared_layer2)

# Create the output layer
department_output_layer = Dense(len(num_department_classes), activation='softmax', name='department_output')(department_hidden_layer)


In [28]:
num_attrition_classes = attrition_df["Attrition"].value_counts()
print(len(num_attrition_classes))

2


In [29]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden_layer = Dense(64, activation='relu')(shared_layer2)

# Create the output layer
attrition_output_layer = Dense(1, activation='sigmoid', name='attrition_output')(attrition_hidden_layer)




In [30]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output_layer, attrition_output_layer])

# Compile the model
model.compile(optimizer=Adam(), loss={'department_output': 'categorical_crossentropy', 'attrition_output': 'binary_crossentropy'},  metrics={
        'department_output': ['accuracy'],
        'attrition_output': ['accuracy']
    })

# Summarize the model
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 42)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │      2,752 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 32)        │      2,080 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 64)        │      2,112 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 64)        │      2,112 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_output   │ (None, 3)         │        195 │ dense_2[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output    │ (None, 1)         │         65 │ dense_3[0][0]     │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 9,316 (36.39 KB)

 Trainable params: 9,316 (36.39 KB)

 Non-trainable params: 0 (0.00 B)

In [31]:
# Debugging
print("Shape of y_train_department_encoded:", y_train_department_encoded.shape)
print("Shape of y_train_attrition_encoded:", y_train_attrition_encoded.shape)

Shape of y_train_department_encoded: (1102, 3)
Shape of y_train_attrition_encoded: (1102, 2)


In [32]:
# Debuggin Continued
print("Data type of y_train_department_encoded:", y_train_department_encoded.dtype)
print("Data type of y_train_attrition_encoded:", y_train_attrition_encoded.dtype)

Data type of y_train_department_encoded: float64
Data type of y_train_attrition_encoded: float64


In [33]:
print("Sample values of y_train_department_encoded:", y_train_department_encoded[:10])
print("Sample values of y_train_attrition_encoded:", y_train_attrition_encoded[:10])

Sample values of y_train_department_encoded: [[0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]]
Sample values of y_train_attrition_encoded:   (0, 0)	1.0
  (1, 0)	1.0
  (2, 0)	1.0
  (3, 0)	1.0
  (4, 0)	1.0
  (5, 0)	1.0
  (6, 0)	1.0
  (7, 1)	1.0
  (8, 0)	1.0
  (9, 0)	1.0


In [34]:
# More Debugging
print(X_train_scaled.shape)
print(X_test_scaled.shape)
print(y_train_department_encoded.shape)
print(y_train_attrition_encoded.shape)
print(y_test_department_encoded.shape)
print(y_test_attrition_encoded.shape)

(1102, 42)
(368, 42)
(1102, 3)
(1102, 2)
(368, 3)
(368, 2)


In [35]:
# Convert target data to numpy arrays and reshape if necessary
y_train_department_encoded_np = np.array(y_train_department_encoded).reshape(-1, 1)
y_train_attrition_encoded_np = np.array(y_train_attrition_encoded).reshape(-1, 1)
y_test_department_encoded_np = np.array(y_test_department_encoded).reshape(-1, 1)
y_test_attrition_encoded_np = np.array(y_test_attrition_encoded).reshape(-1, 1)



In [36]:
print("X_train_scaled shape:", X_train_scaled.shape)
print("y_train_department_encoded_np shape:", y_train_department_encoded_np.shape)
print("y_train_attrition_encoded_np shape:", y_train_attrition_encoded_np.shape)


X_train_scaled shape: (1102, 42)
y_train_department_encoded_np shape: (3306, 1)
y_train_attrition_encoded_np shape: (1, 1)


In [37]:
# Ensure the number of samples match between input and target data
assert X_train_scaled.shape[0] == y_train_department_encoded_np.shape[0] == y_train_attrition_encoded_np.shape[0], "Training data size mismatch"
assert X_test_scaled.shape[0] == y_test_department_encoded_np.shape[0] == y_test_attrition_encoded_np.shape[0], "Test data size mismatch"

AssertionError: Training data size mismatch

In [ ]:
epochs = 20
batch_size = 32

# Train the model
history = model.fit(X_train_scaled, {'department_output': y_train_department_encoded_np, 'attrition_output': y_train_attrition_encoded_np}, 
                    validation_data=(X_test_scaled, {'department_output': y_test_department_encoded_np, 'attrition_output': y_test_attrition_encoded_np}), 
                    epochs=epochs, batch_size=batch_size, verbose=2)


In [ ]:
# Evaluate the model with the testing data
evaluation = model.evaluate(X_test_scaled,[y_test_department_encoded, y_test_attrition_encoded])

# Print the full evaluation results
print("Results:", evaluation)

In [ ]:
# Print the accuracy for both department and attrition
print("department accuracy",evaluation[1])
print("attrition accuracy",evaluation[2])

# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?
   Accuracy is usually a good metric to use on data, but in this case the value may not accurately reflect the data. There is a large imbalance for attrition that would make choosing even just one side all the time fairly accurate. One could test this using precision metrics instead.

3. What activation functions did you choose for your output layers, and why?

I used a softmax for department accuracy because there are multiple outcomes beyond just 2. For attrition I used a sigmoid because there are only two outcomes.

5. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. 
2. 
3. 